In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [129]:
from openai import OpenAI, APIError, BadRequestError

In [4]:
import os

In [5]:
import chromadb

In [6]:
chroma_client = chromadb.Client()

In [7]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [178]:
collection = chroma_client.create_collection(name = "gita_collection_1")

In [269]:
krishna_solution_1 = chroma_client.create_collection(name = "krishna_collection_2")

In [92]:
import json

In [252]:
from tqdm import tqdm

In [108]:
json_dir = '/Users/Rakshit.Lodha/Desktop/talktokrishna/archive/slok'

In [233]:
cleaned_data = []

for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(json_dir, filename)
    with open(file_path,'r') as f:
        data = json.load(f)

        chapter = data.get("chapter")
        verse = data.get("verse")
        slok = data.get("slok")
        prabhu = data.get("prabhu",{})
        translation = prabhu.get("et","")
        commentary = prabhu.get("ec", "")

        

        cleaned_data.append({
            "chapter": chapter,
            "verse": verse,
            "slok": slok,
            "translation": translation,
            "commentary": commentary
        })

In [239]:
cleaned_data[0]

{'chapter': 17,
 'verse': 25,
 'slok': 'तदित्यनभिसन्धाय फलं यज्ञतपःक्रियाः |\nदानक्रियाश्च विविधाः क्रियन्ते मोक्षकाङ्क्षिभिः ||१७-२५||',
 'translation': 'Without desiring fruitive results, one should perform various kinds of sacrifice, penance and charity with the word tat. The purpose of such transcendental activities is to get free from material entanglement.',
 'commentary': ' To be elevated to the spiritual position, one should not act for any material gain. Acts should be performed for the ultimate gain of being transferred to the spiritual kingdom, back to home, back to Godhead.'}

In [263]:
krishna_verses = []

for verse_dict in cleaned_data:
    if verse_dict['chapter'] > 2 and verse_dict['commentary'] != 'There is no purport for this verse' and verse_dict['chapter'] != 11:
        krishna_verses.append(verse_dict)

In [265]:
translation_data = []

for v in krishna_verses:
    translation = v['translation']
    translation_data.append(translation)

In [266]:
def create_embeddings(text):
    embeddings = client.embeddings.create(
        input = text, 
        model = "text-embedding-3-small"
    )

    response = embeddings.data[0].embedding
    return response

In [267]:
final_verse_emb = []

for i in tqdm(translation_data, desc = "Creating Embeddings"):
    try:
        verse_emb = create_embeddings(i)
        final_verse_emb.append(verse_emb)
    except BadRequestError as e:
        print(f"Skipping this verse due to invalid input (400 Bad request): {e}")
        continue

Creating Embeddings: 100%|████████████████████| 525/525 [04:31<00:00,  1.94it/s]


In [271]:
final_id = []

for index, element in enumerate(translation_data, start = 1):
    ids = f"id_{index}"
    final_id.append(ids)

In [272]:
final_metadata = []

for x in krishna_verses:
    final_metadata.append(
        {
            "chapter": x["chapter"],
            "verse": x["verse"],
            "slok": x["slok"],
            "commentary": x["commentary"]
        }
    )

In [273]:
krishna_solution_1.add(
    ids = final_id,
    documents = translation_data,
    embeddings = final_verse_emb,
    metadatas = final_metadata
)

In [291]:
def query():
    # query = input("Whats your issue?")
    query = "My GF left me"
    input_conversion = create_embeddings(query)
    final_result = krishna_solution_1.query(
        query_embeddings = [input_conversion],
        n_results = 10
    )

    final = final_result['documents']

    context = ""

    for i in range(10):
        verse_text = final[0][i]
        context += verse_text + '\n\n'

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
            You are krishna and the user has asked you a query: {query}, your job is to look at the entire {context} of verses.

            Choose a maximum of 3 verses, which you think are relavant to the {query}. You may choose only 1 as well, if you think, the others
            are not relavant. 

            Your job is to then give an empathetic reply to the user, starting with: Hey Vats, I understand

            And take those 1-3 verses and structurally put them in the response you give. 
            """}
        ], temperature = 0.7
    )

    
    return print(response.choices[0].message.content)

In [292]:
query()

Hey Vats, I understand that going through a breakup can be incredibly painful and disheartening. During times like these, it's important to remember that love and devotion can take many forms, and connecting with a higher purpose can provide solace. 

As it is stated, "If one offers Me with love and devotion a leaf, a flower, a fruit or water, I will accept it." This highlights that even the smallest gestures of love and sincerity are valued. It's a reminder that you are not alone in your feelings and that you can offer your heart to something greater.

In moments of heartbreak, it can be helpful to reflect on the mind. "For him who has conquered the mind, the mind is the best of friends; but for one who has failed to do so, his mind will remain the greatest enemy." This suggests that while your emotions may feel overwhelming, mastering your thoughts can lead to healing and strength.

Lastly, remember that every ending can lead to a new beginning. "Abandon all varieties of religion and

In [282]:
final = [['There is no work that affects Me; nor do I aspire for the fruits of action. One who understands this truth about Me also does not become entangled in the fruitive reactions of work.',
  'The worker who is attached to work and the fruits of work, desiring to enjoy those fruits, and who is greedy, always envious, impure, and moved by joy and sorrow, is said to be in the mode of passion.',
  'The worker who is always engaged in work against the injunctions of the scripture, who is materialistic, obstinate, cheating and expert in insulting others, and who is lazy, always morose and procrastinating is said to be a worker in the mode of ignorance.',
  'O Dhanañjaya, all this work cannot bind Me. I am ever detached from all these material activities, seated as though neutral.',
  'It is better to engage in one’s own occupation, even though one may perform it imperfectly, than to accept another’s occupation and perform it perfectly. Duties prescribed according to one’s nature are never affected by sinful reactions.',
  'If, however, you are unable to work in this consciousness of Me, then try to act giving up all results of your work and try to be self-situated.',
  'I am the only enjoyer and master of all sacrifices. Therefore, those who do not recognize My true transcendental nature fall down.',
  'Perform your prescribed duty, for doing so is better than not working. One cannot even maintain one’s physical body without work.',
  'The intelligent renouncer situated in the mode of goodness, neither hateful of inauspicious work nor attached to auspicious work, has no doubts about work.',
  'O son of Pṛthā, there is no work prescribed for Me within all the three planetary systems. Nor am I in want of anything, nor have I a need to obtain anything – and yet I am engaged in prescribed duties.']]


context = ""

for i in range(10):
    verse_text = final[0][i]
    context += verse_text + "\n\n"

print(context)

There is no work that affects Me; nor do I aspire for the fruits of action. One who understands this truth about Me also does not become entangled in the fruitive reactions of work.

The worker who is attached to work and the fruits of work, desiring to enjoy those fruits, and who is greedy, always envious, impure, and moved by joy and sorrow, is said to be in the mode of passion.

The worker who is always engaged in work against the injunctions of the scripture, who is materialistic, obstinate, cheating and expert in insulting others, and who is lazy, always morose and procrastinating is said to be a worker in the mode of ignorance.

O Dhanañjaya, all this work cannot bind Me. I am ever detached from all these material activities, seated as though neutral.

It is better to engage in one’s own occupation, even though one may perform it imperfectly, than to accept another’s occupation and perform it perfectly. Duties prescribed according to one’s nature are never affected by sinful reac